In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [515]:
#Dados brasil.io
covid = pd.read_csv('caso_full.csv')
#conjunto de dados por regiao
norte = covid[(covid.state.isin(['AM','RR','AP','PA','TO','RO','AC'])) &
              (covid.place_type == 'state')].sort_values(['date'],ascending = True).groupby(
    ['epidemiological_week']).new_confirmed.sum()

nordeste = covid[covid.state.isin((['MA','PI','CE','RN','PE','PB','SE','AL','BA'])) &
              (covid.place_type == 'state')].sort_values(['date'],ascending = True).groupby(
    ['epidemiological_week']).new_confirmed.sum()

centrooeste = covid[(covid.state.isin(['MT','MS','GO'])) &
              (covid.place_type == 'state')].sort_values(['date'],ascending = True).groupby(
    ['epidemiological_week']).new_confirmed.sum()

sudeste = covid[covid.state.isin((['SP','RJ','ES','MG'])) &
              (covid.place_type == 'state')].sort_values(['date'],ascending = True).groupby(
    ['epidemiological_week']).new_confirmed.sum()

sul = covid[covid.state.isin((['PR','RS','SC'])) &
              (covid.place_type == 'state')].sort_values(['date'],ascending = True).groupby(
    ['epidemiological_week']).new_confirmed.sum()


#dadosFVS-AM
df_am = pd.read_csv('casos_am2.csv')
df_am = df_am.drop(columns=['Unnamed: 0'])
df_am['Obitos'] = df_am['Obitos'].astype(int)
df_am['Confirmados'] = df_am['Confirmados'].astype(int)
df_am['Incidencia'] = df_am['Incidencia'].astype(float)
#isolar dados da capital e interior
df_am = df_am[(df_am.Municipio.isin(['Manaus','INTERIOR']))]
df_am['Data'] = pd.to_datetime(df_am['Data'])

#csv de obitos da fvs
df_obitos = pd.read_csv('obitos.csv')
df_obitos = df_obitos.drop(columns=['Unnamed: 0'])
df_obitos['Data'] = pd.to_datetime(df_obitos['Data'])

In [487]:
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.models import FactorRange
output_notebook()

Loading BokehJS ...

In [520]:
def geraGrafico(df,regiao,cor):
    source = ColumnDataSource(data=dict(semana=df['semana'], casos=df['casos']))
    TOOLTIPS = [
        ("semana", "@semana"),
        ("casos", "@casos"),
    ]
    bar_chart = figure(title=regiao, x_axis_label='Semana Epidemiológica', y_axis_label='Casos', plot_height=300, tooltips=TOOLTIPS)
    bar_chart.vbar(x='semana', top='casos', color=cor, width=0.5, source=source)
    #show(bar_chart)
    return bar_chart

def geraGraficoInterior(x_lb,y_lb,titulo,y1,y2):
    p = figure(plot_width=600, plot_height=250, x_axis_type="datetime", title=titulo,
              x_axis_label=x_lb, y_axis_label=y_lb)
    p.line(df_am['Data'], y1, color='red', alpha=0.5, line_width=4, legend_label='Manaus')
    p.line(df_am['Data'], y2, color='green', alpha=0.5, line_width=4, legend_label='Interior')
    p.legend.location = 'top_left'
    return p

def geraGraficoObitos(x_lb,y_lb,titulo,y1):
    p = figure(plot_width=600, plot_height=250, x_axis_type="datetime", title=titulo,
              x_axis_label=x_lb, y_axis_label=y_lb)
    p.line(df_obitos['Data'], y1, color='red', alpha=0.5, line_width=3)
    return p

In [523]:
#Capturo a imagem dos gráficos de cada região
d = {'semana':pd.Series(norte.index).array , 'casos': pd.Series(norte).array}
df = pd.DataFrame(data=d)
nt = geraGrafico(df,'Região Norte','green')

d = {'semana':pd.Series(nordeste.index).array , 'casos': pd.Series(nordeste).array}
df = pd.DataFrame(data=d)
nd = geraGrafico(df,'Região Nordeste','orange')

d = {'semana':pd.Series(centrooeste.index).array , 'casos': pd.Series(centrooeste).array}
df = pd.DataFrame(data=d)
co = geraGrafico(df,'Região Centro-Oeste','blue')

d = {'semana':pd.Series(sudeste.index).array , 'casos': pd.Series(sudeste).array}
df = pd.DataFrame(data=d)
sd = geraGrafico(df,'Região Sudeste','red')

d = {'semana':pd.Series(sul.index).array , 'casos': pd.Series(sul).array}
df = pd.DataFrame(data=d)
sl = geraGrafico(df,'Região Sul','black')


#Ajuste dos tamanho
nt.plot_width = nt.plot_width = 800
nd.plot_width = nd.plot_width = 800
co.plot_width = co.plot_width = 800
sd.plot_width = sd.plot_width = 800
sl.plot_width = sl.plot_width = 800

#Criar um painel para cada regiao
pnordeste = Panel(child=nt, title='Norte')
pnorte = Panel(child=nd, title='Nordeste')
pcentrooeste = Panel(child=co, title='Centro-Oeste')
psudeste = Panel(child=sd, title='Sudeste')
psul = Panel(child=sl, title='Sul')

tabs = Tabs(tabs=[pnordeste, pnorte,pcentrooeste,psudeste,psul])


#Comparativos entre capital e interior
y1 = df_am[df_am.Municipio == 'Manaus'].Incidencia
y2 = df_am[df_am.Municipio == 'INTERIOR'].Incidencia
gr1 = geraGraficoInterior('Dia','Incidencia','Comparação da incidência (casos/1000) de Manaus e Interior',y1,y2)

y1 = df_am[df_am.Municipio == 'Manaus'].Confirmados
y2 = df_am[df_am.Municipio == 'INTERIOR'].Confirmados
gr2 = geraGraficoInterior('Dia','Casos','Crescimento de nº de casos Manaus x Interior',y1,y2)


y1 = df_am[df_am.Municipio == 'Manaus'].Obitos
y2 = df_am[df_am.Municipio == 'INTERIOR'].Obitos
gr3 = geraGraficoInterior('Dia','Obitos','Evolução do nº de Obitos em Manaus e Interior',y1,y2)

#Obitos por dia a cada 24h
y1 = df_obitos.Ultimas24h
gr4 = geraGraficoObitos('Dia','Obitos 24h','Obitos nas ultimas 24h',y1)

grid = gridplot([gr1, gr2, gr3,gr4],ncols=2, plot_width=400, plot_height=300)


In [524]:
show(tabs)
show(grid)